In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import load_model
from keras.applications.mobilenet import MobileNet, decode_predictions,preprocess_input
# from keras.applications.inception_v3 import InceptionV3, decode_predictions,preprocess_input
import cv2
from io import BytesIO
import numpy as np
import urllib
from PIL import Image

Using TensorFlow backend.


In [2]:
img_width, img_height = 224, 224
train_data_dir = "/home/asus-pc/workspace/jyu/deeplearning/notebooks/deep_neural_network/input/emotion/fer2013_ready/train"
validation_data_dir = "/home/asus-pc/workspace/jyu/deeplearning/notebooks/deep_neural_network/input/emotion/fer2013_ready/validation"
batch_size = 32
epochs = 50

In [3]:
nb_train_samples = 20964
nb_validation_samples = 300

In [4]:
model = applications.mobilenet.MobileNet(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

model.summary()












_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       

In [5]:
# Freeze the layers which you don't want to train. Here the first 5 layers are frosen.
for layer in model.layers[:5]:
    layer.trainable = False

In [6]:
#Adding custom Layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# creating the final model
model_final = Model(input = model.input, output = predictions)

/home/asus-pc/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [8]:
# compile the model
learning_rate = 0.0001
# decay_rate = learning_rate / epochs


model_final.compile(loss = "categorical_crossentropy",
optimizer = optimizers.adam(lr=learning_rate),
metrics=["accuracy"])

In [9]:
# Initiate the train and test generators with data Augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

test_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = "categorical",
    color_mode="rgb")

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical",
    color_mode="rgb")

Found 20964 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [10]:
label_map = (train_generator.class_indices)
label_map

{'happy': 0, 'neutral': 1, 'sad': 2}

In [11]:
# Save the model according to the conditions
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10,
verbose=1, mode='auto')
# Fit the new final layers for the model
model_final.fit_generator(
    train_generator,
    samples_per_epoch = nb_train_samples,
    nb_epoch = epochs,
    validation_data = validation_generator,
    nb_val_samples = nb_validation_samples)

# ,    callbacks = [early]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/asus-pc/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/home/asus-pc/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=655, epochs=50, validation_steps=300)`
  # Remove the CWD from sys.path while we load stuff.




Epoch 1/50
655/655 [==============================] - 284s 434ms/step - loss: 1.2505 - acc: 0.3816 - val_loss: 1.0876 - val_acc: 0.3403
Epoch 2/50
655/655 [==============================] - 302s 461ms/step - loss: 0.9482 - acc: 0.5011 - val_loss: 0.9763 - val_acc: 0.4973
Epoch 3/50
655/655 [==============================] - 301s 459ms/step - loss: 0.8413 - acc: 0.5708 - val_loss: 0.9193 - val_acc: 0.5484
Epoch 4/50
655/655 [==============================] - 301s 459ms/step - loss: 0.7915 - acc: 0.5931 - val_loss: 0.8884 - val_acc: 0.5526
Epoch 5/50
655/655 [==============================] - 301s 459ms/step - loss: 0.7592 - acc: 0.6037 - val_loss: 0.8700 - val_acc: 0.5608
Epoch 6/50
655/655 [==============================] - 302s 461ms/step - loss: 0.7360 - acc: 0.6146 - val_loss: 0.9113 - val_acc: 0.5339
Epoch 7/50
655/655 [==============================] - 302s 461ms/step - loss: 0.7163 - acc: 0.6220 - val_loss: 0.9063 - val_acc: 0.5521
Epoch 8/50
655/655 [==========================

In [ ]:
# test
im = cv2.resize(cv2.imread('/home/asus-pc/workspace/jyu/deeplearning/notebooks/deep_neural_network/input/emotion/main/happy/039.jpg'), (img_width, img_height))

im = np.expand_dims(im, axis=0).astype(np.float32)
im=preprocess_input(im)
print (im.shape)
out = model_final.predict(im)
model_classes=["happy","neutral", "sad"]
print (model_classes[np.argmax(out)])
print (out)
print ("Probability: ", out[0][np.argmax(out)])

In [ ]:
def show_result(im):
    im = cv2.resize(im, (img_width, img_height))
    im = np.expand_dims(im, axis=0).astype(np.float32)
    im=preprocess_input(im)
    out = model_final.predict(im)
    model_classes=["happy","neutral", "sad"]
    print (model_classes[np.argmax(out)])
    print (out)
    print ("Probability: ", out[0][np.argmax(out)])

def run_visualization(url):
    """Running model on"""
    try:
        resp = urllib.request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        orignal_im = cv2.imdecode(image, cv2.IMREAD_COLOR)
    except IOError:
        print('Cannot retrieve image. Please check url: ' + url)
        return
    print('running model on image %s...' % url)
    show_result(orignal_im)
image_url = 'https://previews.123rf.com/images/nomadsoul1/nomadsoul11612/nomadsoul1161200592/68198658-angry-drunk-man-in-depression-.jpg'
run_visualization(image_url)

In [12]:
model_final.save('/home/asus-pc/workspace/jyu/deeplearning/notebooks/deep_neural_network/models/fer2013_set_adam_50_model.h5')  # creates a HDF5 file 'my_model.h5'